In [ ]:
import pycountry

def get_country_name(alpha2_code):
    try:
        country = pycountry.countries.get(alpha_2=alpha2_code.upper())
        return country.name if country else None
    except LookupError:
        return None
def get_country_code(country_name):
    try:
        # Search by name (case-insensitive)
        country = next(
            c for c in pycountry.countries 
            if c.name.lower() == country_name.lower()
        )
        return country.alpha_2
    except StopIteration:
        return None


In [ ]:
from rdflib import Dataset

d = Dataset(store="Oxigraph")
d.open("./data/book.db/")

In [ ]:
to_change = dict()

for r in d.query(r"""
PREFIX schema: <http://schema.org/>
SELECT ?name
WHERE {
  ?country a schema:Country .
  ?country schema:name ?name .
  FILTER(STRLEN(?name) > 2)
}
"""):
    cc = get_country_code(r.get("name"))
    if(cc != None):
        to_change[cc] = r.get("name")
    
    # print(r.get("name"), ":" , get_country_code(r.get("name")))

In [ ]:
codes = dict()

for r in d.query(r"""
PREFIX schema: <http://schema.org/>
SELECT ?name
WHERE {
  ?country a schema:Country .
  ?country schema:name ?name .
  FILTER(STRLEN(?name) = 2)
}
"""):
    fn = get_country_name(r.get("name"))
    if(cc != None):
        codes[str(r.get("name"))] = fn

In [ ]:
c = set(codes.keys())
t = set(to_change.keys())

# c.difference(t)

In [ ]:
from string import Template


find_tmp = Template(r"""
PREFIX schema: <http://schema.org/>
DELETE {
    ?country schema:name "$cc"
}
INSERT {
  ?country schema:name "$fullname" .
  ?country schema:alternateName "$cc" .
}
WHERE {
  ?country a schema:Country .
  ?country schema:name "$cc".
}
""")

for key, value in to_change.items():
    q = find_tmp.substitute(fullname = value, cc=key)
    d.update(q)

In [ ]:
for r in d.query(r"""
PREFIX schema: <http://schema.org/>
SELECT ?name
WHERE {
  ?country a schema:Country .
  ?country schema:name ?name .
  FILTER(STRLEN(?name) = 2)
}
"""):
    print(r)

In [ ]:
d.close()